
# Práctica 1: Metaheurísticas basadas en trayectorias: Temple Simulado

<center><h3>
    Diego Barreiro Pérez
</h3></center>


## Instrucciones

Esto es **Jupyter Notebook**, un documento que integra código Python en un archivo Markdown.
Esto nos permite ir ejecutando celdas de código poco a poco, así como generar automáticamente un informe bien formateado de la práctica.

Puedes añadir una celda con el botón **"Insert"** de la barra de herramentas, y cambiar su tipo con **"Cell > Cell Type"**

Para ejecutar una celda de código, la seleccionaremos y pulsaremos el botón **"▶ Run"** de la barra de herramentas.
Para pasar el documento a HTML, seleccionaremos **"File > Download as > HTML (.html)"**

Sigue este guión hasta el final. Ejecuta el código proporcionado paso a paso comprendiendo lo que estás haciendo y reflexionando sobre los resultados. Habrá preguntas intercaladas a lo largo del guión, responde a todas ellas en la sección reservada para ese fin: **"Respuestas a los cuestionarios"**. Por favor, no modifiques ninguna linea de código excepto cuando se te pida explícitamente.

No olvides insertar tu **nombre y apellidos** en la celda superior.

## Entrega de la práctica

La fecha límite de entrega será la indicada en el Campus Virtual. La entrega consistirá de un único archivo comprimido con nombre `APELIDOS_NOME_TempleSimulado.zip` que contenga los seguientes ficheros:

 * `APELIDOS_NOME_TempleSimulado.html`: Archivo HTML fruto de la exportación del presente Notebook, con las preguntas respondidas al final del documento.
 * `APELIDOS_NOME_TempleSimulado.ipynb`: Archivo fuente Jupyter Notebook.
 * Archivo de datos de los problema utilizados en la resolución.


## Preliminares adicionales sobre Python para esta práctica


Además de lo visto en la P0, conviene familiarizarse con algunas funciones disponibles en Python que pueden resultarte especialmente útiles más adelante en la realización de esta práctica.


Por ejemplo, puedes generar números aletorios de la siguiente forma utilizando el paquete `random`.

In [1]:
import random

# podemos crear un numero aleatorio entre 1 y 10
numero_aleatorio = random.randint(1, 10)
print(numero_aleatorio)

# y números aleatorios entre 0 y 1 siguiendo también una distribución uniforme
numero_U = random.uniform(0,1)
print (numero_U)

1
0.596801625801437


Puedes generar vectores de números fijos y aleatorios que a su vez estén barajados de manera aleatoria como se ilustra a continuación.

In [2]:
vector = [x for x in range (1,10)]
print("vector fijo", vector)

random.shuffle(vector)
print(vector)

vector_aleatorio = [random.randint(1, 10) for i in range(1,10)]
print("vector aleatorio", vector_aleatorio)

random.shuffle(vector_aleatorio)
print(vector_aleatorio)

vector fijo [1, 2, 3, 4, 5, 6, 7, 8, 9]
[3, 9, 8, 5, 6, 2, 1, 4, 7]
vector aleatorio [3, 7, 3, 5, 5, 5, 6, 4, 3]
[5, 4, 3, 3, 7, 5, 3, 5, 6]


Otro conjunto de funciones importantes son las que vienen del módulo `math`. Puedes encontrar una lista de la funciones disponibles en https://docs.python.org/3/library/math.html. Ponemos algunos ejemplos de uso.

In [3]:
import math 

# número e elevado a la potencia indicada
e = math.exp(1)
print(e)

power2_e = math.exp(2)
print(power2_e)

# ejemplo de potencia
print(math.pow(e, 1))
print(math.pow(e, 2))

# ejemplo del logaritmo natural de base e
base = e
print (math.log(e))
print (math.log(e, base))

2.718281828459045
7.38905609893065
2.718281828459045
7.3890560989306495
1.0
1.0


Finalmente, funciones del módulo `time` te permitirían obtener de manera aproximada tiempos de ejecución de secciones concretas de código. 

In [4]:
import time
start_time = time.time()

sum = 0
for i in range(1000000):
    sum = sum * 1

print("---- %s segundos ----" % (time.time() - start_time))

---- 0.3020143508911133 segundos ----


## El Problema del Viajante de Comercio (VC) con Temple Simulado

El objetivo de esta práctica es modelar e implementar un agente inteligente que sea capaz de resolver el problema del VC mediante la metaheurística (MH) de Enfriamiento o Temple Simulado (SA, del inglés Simulated Annealing). Para ello, realizarás una implementación del algoritmo básico visto en la clase expositiva y valorarás si la introducción de modificaciones en el diseño del algoritmo te permite mejorar la calidad de las soluciones alcanzadas.


### Definición del problema de Viajante de Comercio (VC)



El problema del viajante de comercio (VC) es el problema de la persona que quiere vender un producto, y que para ello quiere encontrar el viaje más corto posible a través de las ciudades de los clientes, haciendo una única visita a cada una, empezando y acabando el recorrido en su propia ciudad (recorrido circular desde la ciudad inicial).

Típicamente, el problema parte de una representación mediante un grafo ponderado G=(N, A), donde N es el conjunto de n=|N| nodos (ciudades), y siendo A el conjunto de arcos conectando los nodos. Cada arco (i, j) ∈ A tiene asignado un peso d_ij que representa la distancia entre las ciudades i y j.


Para facilitar vuestra labor de implementación, os proporcionamos la clase `Localizaciones`, que permite cargar las localizaciones GPS que representan los vértices del grafo G de N ciudades, y permite calcular de manera transparente la distancia entre cualquier par de ciudades usando la fórmula del semiverseno https://es.wikipedia.org/wiki/F%C3%B3rmula_del_semiverseno, que sirve para calcular las distancias teniendo en cuenta la curvatura de la Tierra.


En primer lugar importa el módulo Python que acompaña esta práctica, que trae alguna función de apoyo implementada así como la clase de carga de datos.

In [5]:
from helpers_mod_sa import *

Inspecciona el código de carga de localizaciones mediante `psource(Localizaciones)`

In [6]:
psource (Localizaciones)

Fíjate que por defecto se carga el fichero `./data/grafo8cidades.txt`, que contiene las coordenadas GPS de 8 ciudades gallegas, siendo Santiago de Compostela la primera de ellas. La primera línea de estos ficheros indica el número de ciudades n, mientras que cada una de las líneas sucesivas especifican las coordenadas de cada ciudad, especificadas como coordenadas GPS (latitud y longitud en grados).

Puedes cargar otro fichero haciendo uso del parámetro `filename` como se muestra a continuación. Si todo va bien, la primera distancia entre la ciudad 0 y 1 debe ser unos 55 km.

In [7]:
g1 = Localizaciones(filename='./data/grafo8cidades.txt')
print(g1.distancia(0,1))
g2 = Localizaciones(filename='./data/grafos10_10/grafo_1.txt')
print(g2.distancia(0,1))

55.88273580792048
119.30959564041359


El VC se reduce al problema de crear el circuito Hamiltoniano de longitud mínima sobre el grafo G. La solución a una instancia del problema del VC puede representarse como una permutación de los índices de las ciudades, donde lo importante es el orden de visita, que determinará el coste del viaje en términos de la distancia recorrida total. 

De este modo, el problema pertenece a la categoría de problemas NP, pues puede haber n permutaciones que se corresponden al espacio de búsqueda posible. Esto hace que resolver instancias de problemas con muchas ciudades (n grande) haga el problema impracticable con estrategias de búsqueda no-informadas y éste pueda beneficiarse de ciertas metaheurísticas, pudiendo abordar de problemas con tallas más grande a la vez que se obtienen soluciones razonablemente buenas.


## P1.1: Implementación básica de Temple Simulado



Implementa el algoritmo básico de Temple Simulado para resolver el problema del VC enunciado arriba. Para ello, revisa la descripción algorítmica de la MH vista en la clase expositiva (Véase el T1, diapositiva 37 y asociadas).

Ten en cuenta las siguientes consideraciones de diseño para completar la implementación básica:
- Representación  de  las  soluciones:  representación  de  orden  (permutaciones)  comenzando y finalizando en la ciudad 0. 

- Solución inicial: generación aleatoria de una permutación válida.

- Operador  de  selección  de  la  solución  siguiente  Scand  a  partir  de  la  actual  Sact:  operador  de  intercambio. 

- Función de coste: suma de las distancias del camino según el orden del recorrido.

- Mecanismo de enfriamiento: mecanismo exponencial decreciente que sigue la expressión 𝑇(𝑘)=T0·𝑒(−𝜆𝑘), donde 𝑘 es el número de iteración y T0, λ son los parámetros de diseño.

    Para esta implementación básica considera, 𝑇(𝑘) = 20 · 𝑒(−0.0045𝑘), para establecer los parámetros por defecto de T0 y 𝜆, con temperatura inicial fija T0=20 y 𝜆=0.0045 respectivamente.

- Condición de parada: número fijo de iteraciones (limite=1000). No se precisa definir Tfinal.

Para verificar tu implementación, puedes utilizar el fichero de localizaciones de 8 ciudades gallegas (*grafo8cidades.txt*). La solución óptima resuelta con una búsqueda informada como A* se situa en torno a los 382km.

Para comprobar que tu implementación es suficientemente general como para manejar problemas del VC diferentes tallas, puedes probar también con el fichero de localizaciones de 120 ciudades de USA proporcionado en esta práctica (*US120.txt*).


In [8]:
# REPRESENTACIÓN DE LAS SOLUCIONES
# En este diseño, a pesar de que siempre se empieza en la ciudad 0 y se acaba en la ciudad 0, se
# procede a almacenar todo el orden de las ciudades, incluyendo las iniciales y las finales, para
# facilitar ciertas funciones como coste.
# [0, 1, 2, 3, 4, 5, 6, 7, 0]

# TODO: Confirmar si se almacenan todas las soluciones actuales o solo la última

In [9]:
# SOLUCIÓN INICIAL
def genera_solucion_inicial(loc):
    sol = [x for x in range(1, loc.nciudades)]
    random.shuffle(sol)
    return [0] + sol + [0]

In [10]:
# OPERADOR DE SELECCIÓN DE LA SOLUCIÓN SIGUIENTE
def selecciona_solucion_siguiente(loc, sact):
    # TODO: Confirmar si se selecciona aleatoriamente, o por fuerza bruta para probar todas las combinaciones
    #       en el segundo bucle.
    pos_cambio = random.randint(1, loc.nciudades-1)
    pos_opuesto = pos_cambio
    while pos_opuesto == pos_cambio:
        pos_opuesto = random.randint(1, loc.nciudades-1)
    
    scand = sact.copy()
    scand[pos_cambio], scand[pos_opuesto] = sact[pos_opuesto], sact[pos_cambio]
    return scand

In [11]:
# FUNCIÓN DE COSTE
def coste(loc, sol):
    s = 0
    for i in range(0, len(sol)-1):
        s += loc.distancia(sol[i], sol[i+1])
    return s

In [12]:
# MECANISMO DE ENFRIAMIENTO
t0, l = 20, 0.0045

# NOTA: Se pasan todos estos parámetros "no útiles" para poder mantener la misma función
# de temple_simulado más adelante al definir un valor de T0 dinámico.
T0 = lambda loc, prob, empeorar, sol_inicial: t0

def T(k):
    return 20 * math.exp(-l*k)

In [13]:
# CONDICIÓN DE PARADA
limite = 1000

In [14]:
# TEMPLE SIMULADO
def temple_simulado(loc):
    sact = genera_solucion_inicial(loc)
    t = T0(loc, 0.5, 0.01, sact)
    for i in range(0, limite):
        # TODO: Confirmar si no se hace doble bucle (no se explota)
        scand = selecciona_solucion_siguiente(loc, sact)
        o = coste(loc, scand) - coste(loc, sact)
        # Se inverte el orden de las condiciones (primero si es mejor, luego la "agitación")
        # ya que el cálculo de si es mejor es más sencillo
        if o < 0 or random.uniform(0, 1) < math.exp(-o/t):
            sact = scand

        t = T(i)
    return sact

In [15]:
sol = temple_simulado(g1)
print(coste(g1, sol), sol)
# Las soluciones "óptimas" que salen son [0, 1, 2, 3, 4, 5, 6, 7, 0] y [0, 7, 6, 5, 4, 3, 2, 1, 0]

g3=Localizaciones(filename='./data/US120.txt')
sol = temple_simulado(g3)
print(coste(g3, sol), sol)

381.6699617675482 [0, 7, 6, 5, 4, 3, 2, 1, 0]
84993.4230578482 [0, 59, 105, 91, 22, 109, 41, 24, 71, 8, 100, 90, 43, 118, 85, 12, 67, 88, 86, 61, 33, 97, 57, 6, 44, 106, 115, 19, 5, 72, 54, 112, 27, 23, 119, 78, 26, 63, 46, 108, 31, 93, 18, 56, 111, 49, 20, 42, 107, 13, 69, 84, 37, 89, 83, 113, 58, 74, 2, 79, 68, 47, 95, 116, 29, 4, 21, 11, 14, 50, 28, 53, 40, 38, 101, 10, 114, 51, 36, 3, 52, 103, 25, 70, 9, 15, 99, 64, 62, 117, 92, 34, 17, 60, 35, 82, 75, 55, 7, 39, 1, 66, 65, 104, 48, 76, 87, 98, 96, 73, 102, 45, 30, 16, 94, 110, 81, 32, 77, 80, 0]


❓ **Pregunta 1**. Explica brevemente los detalles relevantes de tu código para entender tu implementación (p.ej., estructura de tu código, funciones, etc.)

❓ **Pregunta 2**. ¿Siempre obtienes soluciones óptimas en cada problema? ¿En qué proporción? Muestra cómo has realizado la verificación y explica brevemente los resultados obtenidos.

Notas: sé conservador en tu estrategia para verificar tu implementación, especialmente cuando empleas ficheros de datos grandes como el del problema de las ciudades USA. Si dejas ejecutando tu algoritmo por un número elevado de iteraciones, puede resultarte útil medir el tiempo que tarda para tomar decisiones sobre donde establecer el límite. 

### Ajuste de temperatura inicial dinámico


En tu implementación has establecido la temperatura inicial siguiendo un valor fijo completamente arbitrario. Esto puede no resultar siempre adecuado, y suele ser más conveniente disponer de un ajuste de la temperatura inicial (T0) que tenga en cuenta la instancia del problema a resolver. 

Modifica la implementación anterior para que puedas parametrizar el valor inicial del parámetro de control T0 utilizando la siguiente fórmula:

    𝑇0 = −𝜇/ln(𝜙)𝐶(𝑆0), que permite aceptar (con una probabilidad 𝜙) soluciones siguientes que empeoren en μ (tanto por uno) a S0 (esto es, que sean (1+ μ) veces peores).

En la fórmula, C(S0) es el coste de la solución inicial, con 𝜙=0.50 y 𝜇=0.01.


In [16]:
# NUEVO MECANISMO DE ENFRIAMIENTO
T0 = lambda loc, prob, empeorar, sol_inicial: -empeorar/math.log(prob)*coste(loc, sol_inicial)

In [17]:
sol = temple_simulado(g1)
print(coste(g1, sol), sol)

sol = temple_simulado(g3)
print(coste(g3, sol), sol)

381.6699617675482 [0, 7, 6, 5, 4, 3, 2, 1, 0]
89663.38952017257 [0, 56, 85, 33, 29, 72, 102, 12, 87, 17, 47, 103, 118, 45, 30, 112, 54, 61, 1, 111, 60, 93, 28, 21, 86, 34, 9, 96, 117, 14, 91, 77, 119, 71, 16, 116, 39, 95, 19, 53, 80, 100, 68, 10, 89, 81, 109, 51, 79, 38, 7, 106, 76, 6, 44, 104, 108, 59, 107, 22, 101, 32, 43, 84, 98, 73, 110, 74, 24, 26, 41, 65, 5, 31, 88, 15, 115, 82, 40, 50, 4, 36, 23, 25, 27, 52, 20, 67, 75, 55, 37, 13, 105, 78, 114, 90, 113, 11, 2, 8, 42, 97, 57, 48, 70, 99, 94, 64, 58, 63, 62, 92, 46, 49, 3, 83, 69, 35, 18, 66, 0]


❓ **Pregunta 3**. Confecciona una tabla que muestre la relación de 𝐶(𝑆0), 𝜙 y 𝜇 con T0 para los dos problemas utilizados anteriormente (ciudades gallegas y ciudades USA). Entonces, de manera razonada amplia la tabla con variaciones de 𝜙 y 𝜇 para uno de los problemas. ¿Qué combinación da mayor/menor T0?

❓ **Pregunta 4**. Escoge 5 combinaciones variadas de parámetros (incluidos los que regulan el mecanismo de enfriamiento, e.g., 𝜆). Ejecuta el algoritmo de Temple Simulado con esas combinaciones. Guarda los resultados en una celda, y discute los resultados obtenidos en cuanto a la calidad de las soluciones obtenidas.



 
## P1.2: Mejoras del algoritmo de Temple Simulado (No obligatorio)

En esta sección el objetivo es realizar mejoras al algoritmo desarrollado previamente, de acuerdo a lo visto en las clases expositivas. Podrá modificarse cualquier parámetro u operador, como por ejemplo:
- La solución inicial
- El valor inicial del parámetro de control (T0).
- El mecanismo de enfriamiento.
- La velocidad de enfriamiento.
- El mecanismo de selección de las soluciones candidatas
- El criterio de parada

❓ **Pregunta 5**. ¿Qué intervenciones de mejora te ha llevado a mejores resultados? Explica brevemente las mejoras o intervenciones de mejora realizadas, cómo la has implementado, porqué las consideras buenas para el problema y presenta tus conclusiones acompañadas de los resultados obtenidos.



#### Respuestas al cuestionario (10 puntos)

Recordatorio: No olvides escribir tu nombre y apellidos en la segunda celda de este documento.
La respuestas a las preguntas deben venir acompañadas de las implementaciones necesarias para su respuesta.

*P1.1: Implementación básica*

Pregunta 1 (3 puntos). 
    Incluye las celdas que consideres oportunas para que sea legible y fácil de seguir.

Pregunta 2 (1 puntos)

Pregunta 3 (0.5 puntos)

Pregunta 4 (1.5 puntos)

*P1.2: Implementación mejoras* (No obligatoria)

Pregunta 5 (4 puntos)